In [157]:
import pandas as pd

In [158]:
FILENAME = "satellite-dev.csv"
df = pd.read_csv(FILENAME, sep=",")

In [159]:
df

,Name,Norad ID,Launch Date,Decay Date,Classification
0,SPUTNIK 1,2,Fri Oct 04 1957 00:00:00 GMT+0100 (Mitteleurop...,Fri Jan 03 1958 01:00:00 GMT+0100 (Mitteleurop...,false
1,SPUTNIK 2,3,Sun Nov 03 1957 00:00:00 GMT+0100 (Mitteleurop...,Mon Apr 14 1958 01:00:00 GMT+0100 (Mitteleurop...,false
2,EXPLORER 1,4,Sat Feb 01 1958 00:00:00 GMT+0100 (Mitteleurop...,Tue Mar 31 1970 01:00:00 GMT+0100 (Mitteleurop...,false
3,VANGUARD 1,5,Mon Mar 17 1958 00:00:00 GMT+0100 (Mitteleurop...,false,Space & Earth Science
4,EXPLORER 3,6,Wed Mar 26 1958 00:00:00 GMT+0100 (Mitteleurop...,Sat Jun 28 1958 01:00:00 GMT+0100 (Mitteleurop...,false
...,...,...,...,...,...
16486,SOLWIND DEB,16488,Sat Feb 24 1979 00:00:00 GMT+0100 (Mitteleurop...,Wed Feb 01 1989 01:00:00 GMT+0100 (Mitteleurop...,false
16487,SOLWIND DEB,16489,Sat Feb 24 1979 00:00:00 GMT+0100 (Mitteleurop...,Mon Apr 17 1989 02:00:00 GMT+0200 (Mitteleurop...,false
16488,COSMOS 1724,16490,Wed Jan 15 1986 00:00:00 GMT+0100 (Mitteleurop...,Sat Mar 15 1986 01:00:00 GMT+0100 (Mitteleurop...,false
16489,SL-4 R/B,16491,Wed Jan 15 1986 00:00:00 GMT+0100 (Mitteleurop...,Tue Jan 21 1986 01:00:00 GMT+0100 (Mitteleurop...,false


In [160]:
launch_dates = df["Launch Date"]
decay_dates = df["Decay Date"]

In [161]:
lds = []
for date in launch_dates:
    lds.append(date.split(" ")[3])
    
dds = []
for date in decay_dates:
    if date == "false":
        dds.append("false")
    else:
        dds.append(date.split(" ")[3])

decay_df = pd.DataFrame({ "Decay Date": dds })
launch_df = pd.DataFrame({ "Launch Date": lds })

ndf = df.copy(deep=True)
ndf.update(decay_df)
ndf.update(launch_df)

In [162]:
columns = ["Classification"] + list(range(1957, 2025))

In [163]:
def accumulate_years(dataframe, classification):
    gdf = dataframe[dataframe["Classification"] == classification]
    
    yearly_numbers = [0] * (2026 - 1957)

    year = 1957
    satellites = 0

    for i in range(0, len(gdf.index)):
        launch_date = int(gdf.iloc[i, 2])
        decay_date = gdf.iloc[i, 3]

        # This year will be a satellite more
        yearly_numbers[launch_date - 1957] += 1

        if decay_date != "false":
            yearly_numbers[int(decay_date) - 1957 + 1] -= 1

    for i in range(0, len(yearly_numbers) - 1):
        yearly_numbers[i + 1] += yearly_numbers[i]
    yearly_numbers.pop()
    data = [classification] + yearly_numbers
    return data

In [164]:
classifiers = []
for key in ndf.groupby(["Classification"]).groups.keys():
    if key != "false":
        classifiers.append(key)
        
years = []
for classifier in classifiers:
    data = accumulate_years(ndf, classifier)
    years.append(data)

In [165]:
years_df = pd.DataFrame(years, columns=columns)
years_df

,Classification,1957,1958,1959,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Amateur radio,0,0,0,0,1,2,0,0,2,...,15,15,15,15,15,15,15,15,15,15
1,Brightest,0,0,0,0,0,0,1,3,3,...,44,42,41,40,40,40,40,39,38,37
2,Earth resources,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7
3,Engineering,0,0,0,0,0,0,0,1,7,...,21,21,21,21,21,21,21,21,21,21
4,Experimental,0,0,0,1,1,1,2,2,4,...,11,11,11,11,11,11,11,11,11,11
5,GOES,0,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
6,Geodetic,0,0,0,0,0,0,0,0,0,...,23,23,23,23,23,23,23,23,23,23
7,Geostationary,0,0,0,0,0,0,2,3,3,...,157,157,157,157,157,157,157,157,157,157
8,Global Positioning System (GPS) Constellation,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
9,Glonass Constellation,0,0,0,0,0,0,0,0,0,...,21,21,21,21,21,21,21,21,21,21
